In [24]:
import re
import json
from timeout_decorator import timeout


In [25]:
with open("./RegexEval_manual_1.json") as f:
    data = json.load(f)

len(data)

3648

In [26]:
@timeout(60)
def match_expression(expression, text):
    try:
        return re.search(expression, text) is not None
    except Exception as e:
        print(e)
        return None

In [ ]:
new_data = []
for item in data:
    if item['expression'].strip() == '' or item['description'].strip() == '' or item['matches'].strip() == '' or item['non_matches'].strip() == '':
        continue
    
    is_valid = True
    expression = item['expression'].strip()
    description = item['description'].strip()
    matches = [x.strip() for x in item['matches'].strip().split('|')]
    non_matches = [x.strip() for x in item['non_matches'].strip().split('|')]


    for match in matches:
        try:
            is_match = match_expression(expression, match)
        except TimeoutError:
            print('TimeoutError: ', expression, match)
            is_match = None
        if is_match is None or is_match == False:
            is_valid = False
            break    
    for non_match in non_matches:
        try:
            is_match = match_expression(expression, non_match)
        except TimeoutError:
            print('TimeoutError: ', expression, match)
            is_match = None
        if is_match is None or is_match == True:
            is_valid = False
            break

    if is_valid:
        new_data.append(item)
    else:
        print('Invalid: ', expression, description, matches, non_matches)


In [29]:
with open('RegexEval_Filtered.json', 'w') as f:
    json.dump(new_data, f)